In [77]:
import json
import glob

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel

In [79]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [103]:
num_qubit = 4

def construct_random_circuit(two_q_gate_depth_post_trans, factor=1.2):
    if not two_q_gate_depth_post_trans:
        d = np.random.randint(1, 6)
        rc = transpile(random_circuit(num_qubit, d, 1, measure=True), backend=backend_noisy, optimization_level=3)

    elif 1 <= two_q_gate_depth_post_trans <= 8:
        d = np.random.randint(1, two_q_gate_depth_post_trans)
        rc = transpile(random_circuit(num_qubit, d, 2, measure=True), backend=backend_noisy, optimization_level=3)
        while rc.depth(filter_function=lambda x: x[0].num_qubits == 2) != two_q_gate_depth_post_trans:
            rc = transpile(random_circuit(num_qubit, d, 2, measure=True), backend=backend_noisy, optimization_level=3)

    elif 9 <= two_q_gate_depth_post_trans <= 18:
        assert two_q_gate_depth_post_trans%2 == 0
        half_depth = two_q_gate_depth_post_trans // 2
        d = np.random.randint(1, half_depth//factor)

        rc1 = transpile(random_circuit(num_qubit, d, 2, measure=False), backend=backend_noisy, optimization_level=3)
        while rc1.depth(filter_function=lambda x: x[0].num_qubits == 2) != half_depth:
            rc1 = transpile(random_circuit(num_qubit, d, 2, measure=False), backend=backend_noisy, optimization_level=3)

        rc2 = transpile(random_circuit(num_qubit, d, 2, measure=False), backend=backend_noisy, optimization_level=3)
        while rc2.depth(filter_function=lambda x: x[0].num_qubits == 2) != half_depth:
            rc2 = transpile(random_circuit(num_qubit, d, 2, measure=False), backend=backend_noisy, optimization_level=3)

        rc = rc1.compose(rc2)
        rc.measure_all()
    else:
        raise NotImplementedError
    return rc

In [108]:
# circuit = transpile(construct_random_circuit(2), backend=backend_noisy, optimization_level=3)
circuit = construct_random_circuit(8)
circuit.draw(fold=-1)

global phase: 0.67118
                                                                                                                                                                                                                                                                                                                                                                  
ancilla_0 -> 0 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               ┌─────────────┐┌────┐               ┌───┐ ┌────────────┐                                          ┌───┐┌──────────────┐┌────┐┌──────────────┐┌────┐┌──────────────┐┌───┐     ┌───┐┌─────────────┐          ┌────┐         ┌──────────┐  ┌───┐   ┌──────────┐         ┌───┐┌─────────────┐┌───┐┌───┐ ┌─────────────┐                  ░ ┌─┐         
      q_1 -> 1 ┤ Rz(0.90505) ├┤ √X ├───────────────┤ X ├─┤ Rz(1.3899) ├──────────────────────────────────────────┤ X ├┤ Rz(-0.53287) ├┤ √X ├┤ Rz(-0.26403) ├┤ √X ├┤ Rz(-0.51757) ├┤ X ├──■──┤ X ├┤ Rz(-2.6089) ├──────────┤ √X ├─────────┤ Rz(-π/2) ├──┤ X ├───┤ Rz(-π/2) ├─────────┤ X ├┤ Rz(0.12466) ├┤ X ├┤ X ├─┤ Rz(-1.9385) ├──────────────────░─┤M├─────────
               └┬────────────┤├────┤  ┌─────────┐  └─┬─┘ └───┬────┬───┘┌──────────────┐          ┌────┐          └─┬─┘└─┬─────────┬──┘├────┤├─────────────┬┘└────┘└──────────────┘└─┬─┘  │  └─┬─┘└─────────────┘          └────┘         └──────────┘  └─┬─┘   └──┬────┬──┘┌───────┐└─┬─┘└────┬────┬───┘└─┬─┘├───┴┐├─────────────┤┌────┐┌─────────┐ ░ └╥┘      ┌─┐
      q_3 -> 2 ─┤ Rz(2.2365) ├┤ √X ├──┤ Rz(π/2) ├────■───────┤ √X ├────┤ Rz(-0.18088) ├──────────┤ √X ├────────────■────┤ Rz(π/2) ├───┤ √X ├┤ Rz(0.22878) ├─────────────────────────┼────┼────┼──────────────────────────────────────────────────────────■────────┤ √X ├───┤ Rz(π) ├──■───────┤ √X ├──────■──┤ √X ├┤ Rz(-2.1546) ├┤ √X ├┤ Rz(π/2) ├─░──╫───────┤M├
               ┌┴────────────┤├────┤┌─┴─────────┴─┐┌────┐┌───┴────┴───┐└────┬───┬─────┘┌─────────┴────┴─────────┐┌───┐  └─────────┘   └────┘└─────────────┘                         │  ┌─┴─┐  │                 ┌───────────────────────┐                         └────┘   └───────┘          └────┘         └────┘└─────────────┘└────┘└─────────┘ ░  ║ ┌─┐   └╥┘
      q_0 -> 3 ┤ Rz(-2.2912) ├┤ √X ├┤ Rz(-1.7935) ├┤ √X ├┤ Rz(1.8389) ├─────┤ X ├──────┤ Rz(-0.761494543969245) ├┤ X ├──────────────────────────────────────────────────────────────■──┤ X ├──■─────────■───────┤ Rz(0.385540801503158) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────░──╫─┤M├────╫─
               └────┬───┬────┘└────┘└─────────────┘└────┘└────────────┘     └─┬─┘      └────────────────────────┘└─┬─┘   ┌────────┐   ┌────┐  ┌──────────┐                             └───┘          ┌─┴─┐     └──────┬──────────┬─────┘   ┌────┐   ┌────────┐                                                                                     ░  ║ └╥┘┌─┐ ║ 
      q_2 -> 4 ─────┤ X ├─────────────────────────────────────────────────────■────────────────────────────────────■─────┤ Rz(-π) ├───┤ √X ├──┤ Rz(3π/4) ├────────────────────────────────────────────┤ X ├────────────┤ Rz(3π/4) ├─────────┤ √X ├───┤ Rz(-π) ├─────────────────────────────────────────────────────────────────────────────────────░──╫──╫─┤M├─╫─
                    └───┘                                                                                                └────────┘   └────┘  └──────────┘                                            └───┘            └──────────┘         └────┘   └────────┘                                                                                     ░  ║  ║ └╥┘ ║ 
          c: 4/══════════════════════════════════════════════════════════

In [110]:
transpile(circuit, backend_noisy, optimization_level=3).depth(lambda x: x[0].num_qubits == 2)

8

In [112]:
def generate_data(generating_func, two_q_gate_depths, save_path: str, n_entries_per_file: int):
    for depth in two_q_gate_depths:
        entries = []

        for i in tqdm_notebook(range(n_entries_per_file)):
            circuit = generating_func(depth)

            job_ideal = execute(circuit, **run_config_ideal)
            job_noisy = execute(circuit, **run_config_noisy)

            counts_ideal = job_ideal.result().get_counts()
            counts_noisy = job_noisy.result().get_counts()

            ideal_exp_val = cal_z_exp(counts_ideal)
            noisy_exp_val = cal_z_exp(counts_noisy)

            graph_data = circuit_to_graph_data_json(
                circuit=circuit,
                properties=properties,
                use_qubit_features=True,
                use_gate_features=True,
            )

            entry = ExpValueEntry(
                circuit_graph=graph_data,
                observable=[],
                ideal_exp_value=ideal_exp_val.tolist(),
                noisy_exp_values=[noisy_exp_val.tolist()],
                circuit_depth=circuit.depth()
            )
            entries.append(entry.to_dict())

        path = f"{save_path}/depth{depth}.json"
        with open(path, "w") as f:
            # pprint(entries)
            json.dump(entries, f)

In [ ]:
depths = np.arange(4, 19, 2)
generate_data(construct_random_circuit, depths, './data/haoran_mbd/random_circuits', 500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [24]:
f = open('./data/haoran_mbl/rc_train/random_circuits_depth4.json')
out = json.load(f)

In [25]:
out[1]['circuit_depth']

36